## Summarize the Global Fishing Watch imagery into hexbins

In [13]:
import os
import arcpy
from arcpy.sa import *
import time

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# Set the workspace to the geodatabase
arcpy.env.workspace = r'E:\gis_lib\hexbins\H3_Hexbins.gdb'

p_type = 'gfw_2020_monthly'
summary_raster = r'e:\analysis\GFW\2020\TIF\global_fish_watch_monthly_fleet_sum_2020.crf'
gdb = r'E:\gis_projects\Global Fishing Watch\Global Fishing Watch.gdb'

# List all feature classes in the geodatabase
feature_classes = arcpy.ListFeatureClasses()
global_summary_layer = r'E:\gis_lib\general\Base_content.gdb\Global_Extent_WGS84'
global_summary_field = 'OBJECTID'
hex_summary_field = 'GRID_ID'
eez_summary_layer = r'E:\gis_lib\general\Base_content.gdb\eez_v12'
eez_summary_field = 'OBJECTID'






In [14]:
# Print the feature classes
for fc in feature_classes:
    gdb_root = os.path.split(gdb)[0]
    fc_short = fc.replace('GenerateTessellation_', '').replace('_wgs84', '')
    print("Processing... " + fc_short)
    print(datetime.datetime.now())
    print(os.path.join(gdb, p_type + '_' + fc))
    # print(gdb_root)
    # print(fc_short)
    with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=fc,
            zone_field=hex_summary_field,
            in_value_raster=summary_raster,
            out_table = os.path.join(gdb, p_type + '_' + fc_short),
            ignore_nodata="DATA",
            statistics_type="SUM",
            process_as_multidimensional="ALL_SLICES",
            percentile_values=90,
            percentile_interpolation_type="AUTO_DETECT",
            circular_calculation="ARITHMETIC",
            circular_wrap_value=360,
            out_join_layer = os.path.join(gdb_root, p_type + '_' + fc_short + '.lyr')
        )

Processing... h3_r6
2024-09-25 15:00:46.771802
E:\gis_projects\Global Fishing Watch\Global Fishing Watch.gdb\gfw_2020_monthly_GenerateTessellation_h3_r6_wgs84


In [ ]:
# arcpy.management.PivotTable(
#     in_table="gfw_2020_annual_summary",
#     fields="OBJECTID_1",
#     pivot_field="Variable",
#     value_field="SUM",
#     out_table=r"E:\gis_projects\Global Fishing Watch\Global Fishing Watch.gdb\gfw_2020_annual_summary_pivot"
# )

# arcpy.management.CopyFeatures(
#     in_features=fc,
#     out_feature_class=os.path.join(gdb, p_type + '_' + fc_short + '_fc'),
#     config_keyword="",
#     spatial_grid_1=None,
#     spatial_grid_2=None,
#     spatial_grid_3=None
# )

# arcpy.management.JoinField(
#     in_data=os.path.join(gdb, p_type + '_' + fc_short + '_fc'),
#     in_field=hex_summary_field,
#     join_table=os.path.join(gdb, p_type + '_' + fc_short),
#     join_field=hex_summary_field,
#     fields="*.SUM",
#     fm_option="NOT_USE_FM",
#     field_mapping=None,
#     index_join_fields="NO_INDEXES"
# )

### Summarize the data for the entire globe

In [7]:
with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=global_summary_layer,
        zone_field=global_summary_field,
        in_value_raster=summary_raster,
        out_table = os.path.join(gdb, p_type + '_global_summary'),
        ignore_nodata="DATA",
        statistics_type="ALL",
        process_as_multidimensional="ALL_SLICES",
        percentile_values=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360,
        out_join_layer = os.path.join(gdb, p_type + '_summary.lyr')
    )

### Summarize fishing activity for the different Exclusive Economic Zones (EEZ)

In [12]:
with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=eez_summary_layer,
        zone_field=eez_summary_field,
        in_value_raster=summary_raster,
        out_table = os.path.join(gdb, p_type + '_eez_summary'),
        ignore_nodata="DATA",
        statistics_type="ALL",
        process_as_multidimensional="ALL_SLICES",
        percentile_values=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360,
        out_join_layer = os.path.join(gdb, p_type + '_eez_summary.lyr')
    )